In [1]:
#import libraries

import pandas as pd

import numpy as np

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import catboost as cb
from catboost import CatBoostRegressor

In [2]:
# Data for training
diamonds = pd.read_csv('./diamonds.csv')

# Data for kaggle
diamonds_test = pd.read_csv('./diamonds_test.csv')

In [3]:
#Drop id and unnamed

def column_drop(x,y):
    x.drop(['Unnamed: 0'], inplace = True, axis = 1)
    x.drop(['index_id'], inplace = True, axis = 1)
    y.drop(['id'], inplace = True, axis = 1)
    
column_drop(diamonds,diamonds_test)

In [4]:
#Function to drop registers with 0 in x, y, z

def drop_0s(x):
    x = x[x['x']!=0]
    x = x[x['y']!=0]
    x = x[x['z']!=0]
    return x

In [5]:
# def proporciones correctas

def uniform(x,y):
    x['uniform'] = x['z']/ x['y']
    y['uniform'] = y['z']/ y['y']

In [6]:
# Function to avoid NaN in uniform column

def fill_na(x,y):
    x['uniform'].fillna(x['depth']/100, inplace = True)
    y['uniform'].fillna(y['depth']/100, inplace = True)

In [7]:
# definition of values to explain better colors

conditions = [
    (diamonds['color'] == 'J'),
    (diamonds['color'] == 'D'),
    (diamonds['color'] == 'E'),
    (diamonds['color'] == 'F'),
    (diamonds['color'] == 'H'),
    (diamonds['color'] == 'I'),
    (diamonds['color'] == 'G')
    ]

values = [1,2,2,2,1,1,1]

In [8]:
# def proporciones correctas testing

conditions_test = [
    (diamonds_test['color'] == 'J'),
    (diamonds_test['color'] == 'D'),
    (diamonds_test['color'] == 'E'),
    (diamonds_test['color'] == 'F'),
    (diamonds_test['color'] == 'H'),
    (diamonds_test['color'] == 'I'),
    (diamonds_test['color'] == 'G')
    ]

values_test = [1,2,2,2,1,1,1]

In [9]:
# Function to select what features we want in the model

def column_sel(x):
    y = x.columns
    for i in y:
        drop_yn = input(f'drop o no drop {i}:')
        if drop_yn == 'y':
            x.drop([i], inplace = True, axis = 1)

In [10]:
# Function to encode categorical features

encode_columns = ['cut','clarity', 'color']

def hot_encoding(x):
    y = x[encode_columns]
    y = pd.get_dummies(y)
    x.drop(encode_columns, inplace = True, axis = 1)
    x = pd.merge(x, y, left_index=True, right_index=True)
    return x

In [11]:
#Creation of the good color feature

diamonds['good_color'] = np.select(conditions, values)
diamonds_test['good_color'] = np.select(conditions_test, values_test)

In [12]:
# Drop of 0s in diamonds dataset

diamonds = drop_0s(diamonds)

In [13]:
#Creation of the uniform feature

uniform(diamonds, diamonds_test)

In [14]:
# Fillna in uniform feature in both datasets

fill_na(diamonds, diamonds_test)

In [15]:
# Feature selection diamonds

column_sel(diamonds)

drop o no drop price:n
drop o no drop carat:n
drop o no drop city:y
drop o no drop depth:y
drop o no drop table:y
drop o no drop x:y
drop o no drop y:y
drop o no drop z:y
drop o no drop cut:n
drop o no drop color:n
drop o no drop clarity:n
drop o no drop good_color:n
drop o no drop uniform:n


In [16]:
diamonds

,price,carat,cut,color,clarity,good_color,uniform
0,4268,1.21,Premium,J,VS2,1,0.625920
1,4839,1.20,Premium,J,VS2,1,0.606676
2,368,0.30,Premium,J,VS2,1,0.602740
3,5053,1.20,Premium,J,VS2,1,0.629464
4,3593,1.05,Premium,J,VS2,1,0.597264
...,...,...,...,...,...,...,...
40450,2632,0.59,Very Good,F,IF,2,0.597806
40451,9972,1.01,Very Good,F,IF,2,0.628705
40452,886,0.30,Very Good,F,IF,2,0.621810
40453,3205,0.71,Fair,F,IF,2,0.590361


In [17]:
# Feature selection diamonds test

column_sel(diamonds_test)

drop o no drop carat:n
drop o no drop cut:n
drop o no drop color:n
drop o no drop clarity:n
drop o no drop depth:y
drop o no drop table:y
drop o no drop x:y
drop o no drop y:y
drop o no drop z:y
drop o no drop city:y
drop o no drop good_color:n
drop o no drop uniform:n


In [18]:
diamonds_test

,carat,cut,color,clarity,good_color,uniform
0,0.79,Very Good,F,SI1,2,0.623090
1,1.20,Ideal,J,VS1,1,0.606676
2,1.57,Premium,H,SI1,1,0.624317
3,0.90,Very Good,F,SI1,2,0.636215
4,0.50,Very Good,F,VS1,2,0.626719
...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,2,0.620301
13481,0.71,Ideal,I,VS2,1,0.621291
13482,0.70,Ideal,F,VS1,2,0.618214
13483,0.70,Very Good,F,SI2,2,0.585739


In [19]:
# Encoding categorical features

diamonds = hot_encoding(diamonds)
diamonds_test = hot_encoding(diamonds_test)

-------------------------------------------------------------------------------------------------------------------

**Model**

In [20]:
#we separate features from target

X = diamonds.drop(['price'], axis = 1)
y = diamonds['price']

In [21]:
#scaling

scaler = RobustScaler()
X = scaler.fit_transform(X)
X

array([[ 0.796875  ,  0.        ,  0.52831587, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.78125   ,  0.        , -0.71125351, ...,  0.        ,
         0.        ,  1.        ],
       [-0.625     ,  0.        , -0.96482223, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.625     ,  1.        ,  0.26353222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.015625  ,  1.        , -1.76214267, ...,  0.        ,
         0.        ,  0.        ],
       [-0.515625  ,  1.        , -5.94274325, ...,  0.        ,
         0.        ,  0.        ]])

In [22]:
#Parameters to be tried in cross validation

parameters = {
    'iterations'    : [30, 50, 100,1000,5000]   
}

In [23]:
# Model definition

model = CatBoostRegressor()

In [24]:
#GridSearchCV to look for best parameters

cv = GridSearchCV(model,parameters,cv=5,verbose=3,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-1)
cv.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Learning rate set to 0.5
0:	learn: 2352.6812124	total: 71ms	remaining: 3.48s
1:	learn: 1577.0905781	total: 79.8ms	remaining: 1.91s
2:	learn: 1228.0624620	total: 87.7ms	remaining: 1.37s
3:	learn: 1048.1497587	total: 95.7ms	remaining: 1.1s
4:	learn: 943.6683755	total: 104ms	remaining: 933ms
5:	learn: 871.4806929	total: 110ms	remaining: 804ms
6:	learn: 806.6506467	total: 115ms	remaining: 706ms
7:	learn: 769.5812217	total: 120ms	remaining: 631ms
8:	learn: 733.3741580	total: 129ms	remaining: 586ms
9:	learn: 712.1913637	total: 135ms	remaining: 539ms
10:	learn: 694.8279221	total: 141ms	remaining: 498ms
11:	learn: 687.6452707	total: 146ms	remaining: 462ms
12:	learn: 674.3193893	total: 152ms	remaining: 432ms
13:	learn: 660.4571833	total: 157ms	remaining: 404ms
14:	learn: 654.7251767	total: 164ms	remaining: 382ms
15:	learn: 632.1394304	total: 169ms	remaining: 360ms
16:	learn: 624.8645230	total: 177ms	remaining: 344ms
17:	learn: 620.7412

Learning rate set to 0.5
0:	learn: 2331.8251778	total: 70.9ms	remaining: 3.47s
1:	learn: 1516.8800555	total: 80.7ms	remaining: 1.94s
2:	learn: 1112.4624922	total: 88.9ms	remaining: 1.39s
3:	learn: 929.4851987	total: 99ms	remaining: 1.14s
4:	learn: 829.6661447	total: 107ms	remaining: 959ms
5:	learn: 775.0672708	total: 111ms	remaining: 817ms
6:	learn: 733.4633668	total: 116ms	remaining: 715ms
7:	learn: 707.9988937	total: 122ms	remaining: 640ms
8:	learn: 691.9722077	total: 127ms	remaining: 579ms
9:	learn: 679.7217474	total: 135ms	remaining: 540ms
10:	learn: 670.6315870	total: 144ms	remaining: 512ms
11:	learn: 658.6494914	total: 149ms	remaining: 472ms
12:	learn: 652.2364274	total: 156ms	remaining: 443ms
13:	learn: 640.3645801	total: 165ms	remaining: 423ms
14:	learn: 632.3160245	total: 170ms	remaining: 397ms
15:	learn: 622.6629052	total: 177ms	remaining: 375ms
16:	learn: 613.2738740	total: 182ms	remaining: 352ms
17:	learn: 609.6459619	total: 189ms	remaining: 335ms
18:	learn: 605.1131433	tot

253:	learn: 534.5142283	total: 1.79s	remaining: 5.27s
254:	learn: 534.1932819	total: 1.8s	remaining: 5.26s
255:	learn: 533.8097778	total: 1.81s	remaining: 5.26s
256:	learn: 533.4649075	total: 1.82s	remaining: 5.25s
257:	learn: 533.2295589	total: 1.82s	remaining: 5.24s
258:	learn: 533.0653913	total: 1.83s	remaining: 5.23s
259:	learn: 532.7531517	total: 1.83s	remaining: 5.22s
260:	learn: 532.5110496	total: 1.84s	remaining: 5.22s
261:	learn: 532.3425543	total: 1.85s	remaining: 5.21s
262:	learn: 532.1467214	total: 1.86s	remaining: 5.21s
263:	learn: 531.7994481	total: 1.86s	remaining: 5.2s
264:	learn: 531.5471514	total: 1.87s	remaining: 5.19s
265:	learn: 531.3030512	total: 1.88s	remaining: 5.18s
266:	learn: 531.1329159	total: 1.89s	remaining: 5.17s
267:	learn: 530.8550111	total: 1.89s	remaining: 5.16s
268:	learn: 530.6995280	total: 1.9s	remaining: 5.15s
269:	learn: 530.2896760	total: 1.9s	remaining: 5.14s
270:	learn: 530.0725257	total: 1.91s	remaining: 5.14s
271:	learn: 529.8842699	total: 1

150:	learn: 901.4044360	total: 1.04s	remaining: 33.5s
151:	learn: 897.9383637	total: 1.05s	remaining: 33.5s
152:	learn: 893.1584786	total: 1.05s	remaining: 33.4s
153:	learn: 889.9265455	total: 1.06s	remaining: 33.4s
154:	learn: 886.6278872	total: 1.07s	remaining: 33.3s
155:	learn: 883.0535791	total: 1.08s	remaining: 33.4s
156:	learn: 880.2310697	total: 1.08s	remaining: 33.4s
157:	learn: 877.0851406	total: 1.09s	remaining: 33.3s
158:	learn: 874.0007473	total: 1.09s	remaining: 33.3s
159:	learn: 870.7431749	total: 1.1s	remaining: 33.3s
160:	learn: 868.1696620	total: 1.11s	remaining: 33.3s
161:	learn: 865.1931279	total: 1.11s	remaining: 33.2s
162:	learn: 861.7320345	total: 1.12s	remaining: 33.2s
163:	learn: 858.6988497	total: 1.12s	remaining: 33.2s
164:	learn: 854.9655082	total: 1.13s	remaining: 33.2s
165:	learn: 852.3983310	total: 1.14s	remaining: 33.2s
166:	learn: 849.0750614	total: 1.15s	remaining: 33.2s
167:	learn: 845.7801121	total: 1.15s	remaining: 33.2s
168:	learn: 842.2272655	total

454:	learn: 582.3968024	total: 3.37s	remaining: 33.6s
455:	learn: 582.1935285	total: 3.37s	remaining: 33.6s
456:	learn: 581.7789999	total: 3.38s	remaining: 33.6s
457:	learn: 581.5458209	total: 3.39s	remaining: 33.6s
458:	learn: 581.3431236	total: 3.4s	remaining: 33.6s
459:	learn: 581.1313014	total: 3.4s	remaining: 33.6s
460:	learn: 580.8905263	total: 3.41s	remaining: 33.6s
461:	learn: 580.2647192	total: 3.42s	remaining: 33.6s
462:	learn: 579.9174914	total: 3.43s	remaining: 33.6s
463:	learn: 579.7116150	total: 3.44s	remaining: 33.6s
464:	learn: 579.5599344	total: 3.45s	remaining: 33.6s
465:	learn: 579.3333262	total: 3.46s	remaining: 33.6s
466:	learn: 579.0126116	total: 3.46s	remaining: 33.6s
467:	learn: 578.6473501	total: 3.47s	remaining: 33.6s
468:	learn: 578.1991078	total: 3.48s	remaining: 33.6s
469:	learn: 577.9591384	total: 3.49s	remaining: 33.6s
470:	learn: 577.6981628	total: 3.49s	remaining: 33.6s
471:	learn: 577.5358911	total: 3.5s	remaining: 33.6s
472:	learn: 577.0843121	total: 

...

518:	learn: 491.0996021	total: 1.35s	remaining: 1.25s
519:	learn: 490.8945391	total: 1.36s	remaining: 1.25s
520:	learn: 490.8080554	total: 1.36s	remaining: 1.25s
521:	learn: 490.7213302	total: 1.36s	remaining: 1.25s
522:	learn: 490.6448227	total: 1.36s	remaining: 1.24s
523:	learn: 490.4914676	total: 1.37s	remaining: 1.24s
524:	learn: 490.3471312	total: 1.37s	remaining: 1.24s
525:	learn: 490.2560878	total: 1.37s	remaining: 1.24s
526:	learn: 490.1913945	total: 1.37s	remaining: 1.23s
527:	learn: 490.1112537	total: 1.38s	remaining: 1.23s
528:	learn: 489.9456707	total: 1.38s	remaining: 1.23s
529:	learn: 489.8794668	total: 1.38s	remaining: 1.23s
530:	learn: 489.8440918	total: 1.38s	remaining: 1.22s
531:	learn: 489.7562651	total: 1.39s	remaining: 1.22s
532:	learn: 489.7074813	total: 1.39s	remaining: 1.22s
533:	learn: 489.6005078	total: 1.39s	remaining: 1.22s
534:	learn: 489.4969783	total: 1.4s	remaining: 1.21s
535:	learn: 489.2899998	total: 1.4s	remaining: 1.21s
536:	learn: 489.1981992	total:

673:	learn: 479.1835056	total: 1.74s	remaining: 842ms
674:	learn: 479.1086664	total: 1.74s	remaining: 839ms
675:	learn: 479.0662520	total: 1.75s	remaining: 836ms
676:	learn: 478.9843179	total: 1.75s	remaining: 834ms
677:	learn: 478.9230134	total: 1.75s	remaining: 831ms
678:	learn: 478.8482833	total: 1.75s	remaining: 829ms
679:	learn: 478.7932507	total: 1.75s	remaining: 826ms
680:	learn: 478.7662448	total: 1.76s	remaining: 824ms
681:	learn: 478.6982797	total: 1.76s	remaining: 821ms
682:	learn: 478.6240232	total: 1.76s	remaining: 818ms
683:	learn: 478.5063128	total: 1.77s	remaining: 816ms
684:	learn: 478.4138076	total: 1.77s	remaining: 813ms
685:	learn: 478.2574235	total: 1.77s	remaining: 811ms
686:	learn: 478.1609384	total: 1.77s	remaining: 808ms
687:	learn: 478.1022351	total: 1.78s	remaining: 806ms
688:	learn: 478.0802181	total: 1.78s	remaining: 803ms
689:	learn: 477.9684348	total: 1.78s	remaining: 801ms
690:	learn: 477.9055514	total: 1.78s	remaining: 798ms
691:	learn: 477.8484124	tota

833:	learn: 468.6838957	total: 2.13s	remaining: 424ms
834:	learn: 468.6516718	total: 2.13s	remaining: 421ms
835:	learn: 468.6193173	total: 2.13s	remaining: 419ms
836:	learn: 468.5597553	total: 2.14s	remaining: 416ms
837:	learn: 468.4867143	total: 2.14s	remaining: 414ms
838:	learn: 468.4513785	total: 2.14s	remaining: 411ms
839:	learn: 468.4120166	total: 2.14s	remaining: 409ms
840:	learn: 468.3844880	total: 2.15s	remaining: 406ms
841:	learn: 468.3294126	total: 2.15s	remaining: 403ms
842:	learn: 468.2686231	total: 2.15s	remaining: 401ms
843:	learn: 468.2306833	total: 2.15s	remaining: 398ms
844:	learn: 468.1420453	total: 2.16s	remaining: 396ms
845:	learn: 468.0590337	total: 2.16s	remaining: 393ms
846:	learn: 468.0356843	total: 2.16s	remaining: 391ms
847:	learn: 467.9731820	total: 2.17s	remaining: 388ms
848:	learn: 467.8540723	total: 2.17s	remaining: 386ms
849:	learn: 467.7899987	total: 2.17s	remaining: 383ms
850:	learn: 467.7253160	total: 2.17s	remaining: 381ms
851:	learn: 467.6969547	tota

990:	learn: 459.1325436	total: 2.52s	remaining: 22.9ms
991:	learn: 459.0772955	total: 2.52s	remaining: 20.3ms
992:	learn: 458.9981294	total: 2.52s	remaining: 17.8ms
993:	learn: 458.9492716	total: 2.52s	remaining: 15.2ms
994:	learn: 458.8817287	total: 2.53s	remaining: 12.7ms
995:	learn: 458.8187807	total: 2.53s	remaining: 10.2ms
996:	learn: 458.7794608	total: 2.53s	remaining: 7.62ms
997:	learn: 458.7271909	total: 2.54s	remaining: 5.08ms
998:	learn: 458.6380422	total: 2.54s	remaining: 2.54ms
999:	learn: 458.6165255	total: 2.54s	remaining: 0us


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f7cebc398d0>,
             n_jobs=-1, param_grid={'iterations': [30, 50, 100, 1000, 5000]},
             scoring='neg_root_mean_squared_error', verbose=3)

In [25]:
# Best parameters and score

print('Best hyperparameters: ', cv.best_params_, '\n')
print('Best score: ', -cv.best_score_, '\n')

Best hyperparameters:  {'iterations': 1000} 

Best score:  1082.6386580262617 



In [26]:
#cross_val_score

#scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=5, n_jobs=1)
#scores

In [27]:
# Division train/test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Model definition with best parameters

model = CatBoostRegressor(iterations = 1000)

In [29]:
# Model training

weights = model.fit(X_train, y_train)

Learning rate set to 0.070916
0:	learn: 3745.9324443	total: 3.27ms	remaining: 3.27s
1:	learn: 3520.6240597	total: 6.19ms	remaining: 3.09s
2:	learn: 3315.2264595	total: 9.83ms	remaining: 3.27s
3:	learn: 3121.1096739	total: 13.6ms	remaining: 3.38s
4:	learn: 2947.3784617	total: 16.1ms	remaining: 3.21s
5:	learn: 2781.0053040	total: 19.7ms	remaining: 3.26s
6:	learn: 2630.5386701	total: 23.9ms	remaining: 3.4s
7:	learn: 2489.1456093	total: 26.4ms	remaining: 3.27s
8:	learn: 2361.6586350	total: 29.4ms	remaining: 3.24s
9:	learn: 2243.1602061	total: 31.6ms	remaining: 3.13s
10:	learn: 2132.7914021	total: 34.1ms	remaining: 3.07s
11:	learn: 2037.7042394	total: 36.5ms	remaining: 3s
12:	learn: 1942.6083201	total: 38.8ms	remaining: 2.95s
13:	learn: 1855.4752174	total: 42.4ms	remaining: 2.98s
14:	learn: 1775.0375525	total: 45.9ms	remaining: 3.02s
15:	learn: 1701.3598228	total: 49.2ms	remaining: 3.03s
16:	learn: 1632.4260345	total: 52.2ms	remaining: 3.02s
17:	learn: 1571.0074708	total: 55.1ms	remaining: 

154:	learn: 558.4489308	total: 372ms	remaining: 2.03s
155:	learn: 557.7729522	total: 374ms	remaining: 2.03s
156:	learn: 556.9492730	total: 377ms	remaining: 2.02s
157:	learn: 556.5226979	total: 379ms	remaining: 2.02s
158:	learn: 556.1149991	total: 381ms	remaining: 2.01s
159:	learn: 555.6822886	total: 383ms	remaining: 2.01s
160:	learn: 555.3307199	total: 385ms	remaining: 2.01s
161:	learn: 554.8021475	total: 388ms	remaining: 2s
162:	learn: 554.0611131	total: 390ms	remaining: 2s
163:	learn: 553.6796429	total: 392ms	remaining: 2s
164:	learn: 553.2534084	total: 394ms	remaining: 1.99s
165:	learn: 552.6857750	total: 396ms	remaining: 1.99s
166:	learn: 552.1984638	total: 398ms	remaining: 1.99s
167:	learn: 551.6884526	total: 400ms	remaining: 1.98s
168:	learn: 550.9755971	total: 403ms	remaining: 1.98s
169:	learn: 550.6310263	total: 405ms	remaining: 1.98s
170:	learn: 550.0512694	total: 407ms	remaining: 1.97s
171:	learn: 549.7068195	total: 409ms	remaining: 1.97s
172:	learn: 549.4171764	total: 411ms	

326:	learn: 512.9821146	total: 754ms	remaining: 1.55s
327:	learn: 512.9603204	total: 756ms	remaining: 1.55s
328:	learn: 512.9443070	total: 758ms	remaining: 1.55s
329:	learn: 512.6568930	total: 761ms	remaining: 1.54s
330:	learn: 512.4784044	total: 763ms	remaining: 1.54s
331:	learn: 512.3526532	total: 765ms	remaining: 1.54s
332:	learn: 512.3378427	total: 767ms	remaining: 1.53s
333:	learn: 512.3239999	total: 769ms	remaining: 1.53s
334:	learn: 512.0181656	total: 771ms	remaining: 1.53s
335:	learn: 512.0052066	total: 773ms	remaining: 1.53s
336:	learn: 511.7884867	total: 775ms	remaining: 1.52s
337:	learn: 511.7740814	total: 779ms	remaining: 1.52s
338:	learn: 511.6418124	total: 781ms	remaining: 1.52s
339:	learn: 511.6289305	total: 784ms	remaining: 1.52s
340:	learn: 511.6122198	total: 786ms	remaining: 1.52s
341:	learn: 511.4885644	total: 788ms	remaining: 1.52s
342:	learn: 511.3305018	total: 790ms	remaining: 1.51s
343:	learn: 511.2588841	total: 793ms	remaining: 1.51s
344:	learn: 511.0868084	tota

498:	learn: 492.7654439	total: 1.14s	remaining: 1.14s
499:	learn: 492.7143405	total: 1.14s	remaining: 1.14s
500:	learn: 492.6615374	total: 1.14s	remaining: 1.14s
501:	learn: 492.5139773	total: 1.15s	remaining: 1.14s
502:	learn: 492.4162212	total: 1.15s	remaining: 1.13s
503:	learn: 492.3328871	total: 1.15s	remaining: 1.13s
504:	learn: 492.2429399	total: 1.15s	remaining: 1.13s
505:	learn: 492.1667175	total: 1.16s	remaining: 1.13s
506:	learn: 492.0655042	total: 1.16s	remaining: 1.13s
507:	learn: 491.9332393	total: 1.16s	remaining: 1.12s
508:	learn: 491.8541269	total: 1.16s	remaining: 1.12s
509:	learn: 491.7982462	total: 1.16s	remaining: 1.12s
510:	learn: 491.6540952	total: 1.17s	remaining: 1.12s
511:	learn: 491.5497244	total: 1.17s	remaining: 1.11s
512:	learn: 491.4959554	total: 1.17s	remaining: 1.11s
513:	learn: 491.4544051	total: 1.17s	remaining: 1.11s
514:	learn: 491.4058055	total: 1.18s	remaining: 1.11s
515:	learn: 491.3464320	total: 1.18s	remaining: 1.1s
516:	learn: 491.3242223	total

683:	learn: 476.0695506	total: 1.52s	remaining: 704ms
684:	learn: 475.9807348	total: 1.53s	remaining: 702ms
685:	learn: 475.8770757	total: 1.53s	remaining: 700ms
686:	learn: 475.7862679	total: 1.53s	remaining: 697ms
687:	learn: 475.7093523	total: 1.53s	remaining: 695ms
688:	learn: 475.6799845	total: 1.53s	remaining: 693ms
689:	learn: 475.6583989	total: 1.54s	remaining: 691ms
690:	learn: 475.6202208	total: 1.54s	remaining: 688ms
691:	learn: 475.5912980	total: 1.54s	remaining: 686ms
692:	learn: 475.5708988	total: 1.54s	remaining: 684ms
693:	learn: 475.5568594	total: 1.54s	remaining: 681ms
694:	learn: 475.4963336	total: 1.55s	remaining: 679ms
695:	learn: 475.3496666	total: 1.55s	remaining: 677ms
696:	learn: 475.2813651	total: 1.55s	remaining: 675ms
697:	learn: 475.1841946	total: 1.55s	remaining: 672ms
698:	learn: 475.1263980	total: 1.56s	remaining: 670ms
699:	learn: 475.0326740	total: 1.56s	remaining: 668ms
700:	learn: 474.9005919	total: 1.56s	remaining: 666ms
701:	learn: 474.8071674	tota

878:	learn: 462.6896579	total: 1.91s	remaining: 263ms
879:	learn: 462.6132441	total: 1.91s	remaining: 261ms
880:	learn: 462.5328615	total: 1.91s	remaining: 259ms
881:	learn: 462.4633782	total: 1.92s	remaining: 256ms
882:	learn: 462.3907140	total: 1.92s	remaining: 254ms
883:	learn: 462.3251272	total: 1.92s	remaining: 252ms
884:	learn: 462.2358551	total: 1.92s	remaining: 250ms
885:	learn: 462.1349789	total: 1.93s	remaining: 248ms
886:	learn: 462.0411897	total: 1.93s	remaining: 246ms
887:	learn: 462.0258676	total: 1.93s	remaining: 243ms
888:	learn: 461.9562411	total: 1.93s	remaining: 241ms
889:	learn: 461.8659002	total: 1.93s	remaining: 239ms
890:	learn: 461.8371204	total: 1.94s	remaining: 237ms
891:	learn: 461.7963047	total: 1.94s	remaining: 235ms
892:	learn: 461.7380049	total: 1.94s	remaining: 233ms
893:	learn: 461.6669324	total: 1.94s	remaining: 230ms
894:	learn: 461.6012638	total: 1.95s	remaining: 228ms
895:	learn: 461.4734130	total: 1.95s	remaining: 226ms
896:	learn: 461.4279245	tota

In [30]:
# Model predictions

predictions = model.predict(X_test)

In [31]:
# RMSE

real_error = round(mean_squared_error(y_test, predictions)**0.5, 5)
real_error

529.19606

-------------------------------------------------------------------------------------------------------------------

**Test predictions**

In [32]:
diamonds_test

,carat,good_color,uniform,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,clarity_I1,clarity_IF,...,clarity_VS2,clarity_VVS1,clarity_VVS2,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,0.79,2,0.623090,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1.20,1,0.606676,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.57,1,0.624317,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.90,2,0.636215,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.50,2,0.626719,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,2,0.620301,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
13481,0.71,1,0.621291,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
13482,0.70,2,0.618214,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
13483,0.70,2,0.585739,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
# Diamonds to be predicted

X = diamonds_test

In [34]:
#Scaling

scaler = RobustScaler()
X = scaler.fit_transform(X)
X

array([[ 0.140625  ,  1.        ,  0.33724739, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78125   ,  0.        , -0.71893347, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.359375  ,  0.        ,  0.4161993 , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  1.        ,  0.02346698, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        , -2.066234  , ...,  0.        ,
         0.        ,  0.        ],
       [-0.46875   ,  0.        ,  0.23908483, ...,  0.        ,
         1.        ,  0.        ]])

In [35]:
# Predictions

predictions = model.predict(X)

In [36]:
# Formatting for Kaggle

predictions = pd.DataFrame(predictions, columns = ['Price'])
predictions.index.names = ['id']
predictions

,Price
id,
0,2864.566621
1,5845.157660
2,9746.824453
3,3900.304316
4,1601.774265
...,...
13480,1719.354672
13481,2305.183231
13482,3111.689680


In [37]:
# Creation of file where predictions are saved

predictions.to_csv('./results_catb.csv')